# 单热编码

在第一部分，我们从零起步，帮助小明创建了一个预测冰激淋销量的**多层神经网络**；在第二部分，我们亲手搭建了一个**神经网络训练框架**，支持全连接网络，解决了数值回归任务；在第三部分，我们进一步扩展了框架，使其具备了支持分类任务，处理多维度数据，和局部特征提取（CNN）的能力。

到目前为止，我们还停留在处理静态数据、“瞬时场景”的阶段。模型处理的每一组销量数据，识别的每一张图片，都是彼此独立的个体。

处理静态数据，比如识别一张图片，不需要考虑其他图片是什么内容；而处理另一类数据，比如理解语言、预测股价，并不完全取决于当下说了什么、当前的股价是多少，还必须参考之前说了什么、之前的股价是多少。这类数据的前后元素之间存在强烈的依赖关系，我们称之为**序列数据**（Sequence Data）。

在所有序列数据中，最贴近人类智能、也最复杂的，莫过于语言。

In [1]:
import csv
import re
from abc import abstractmethod, ABC
import numpy as np

np.random.seed(99)

## 基础架构

### 张量

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return np.prod(self.data.shape[1:])

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础数据集

In [3]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size, Tensor(self.labels).size

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

    def estimate(self, predictions):
        pass

## 数据

### IMDB 数据集

In [4]:
class IMDBDataset(Dataset):

    def __init__(self, filename):
        self.filename = filename
        super().__init__()

    def load(self):
        self.reviews = []
        self.sentiments = []
        with open(self.filename, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            for _, row in enumerate(reader):
                self.reviews.append(row[0])
                self.sentiments.append(row[1])

        split_reviews = []
        for line in self.reviews:
            split_reviews.append(self.clean_text(line.lower()).split())

        self.vocabulary = set(word for line in split_reviews for word in line)
        self.word2index = {word: index for index, word in enumerate(self.vocabulary)}
        self.index2word = {index: word for index, word in enumerate(self.vocabulary)}
        self.tokens = [[self.word2index[word] for word in line if word in self.word2index] for line in split_reviews]

    @staticmethod
    def clean_text(text):
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return text

    def train(self):
        self.features = [list(set(index)) for index in self.tokens[:-10]]
        self.labels = [0 if index == "negative" else 1 for index in self.sentiments[:-10]]

    def eval(self):
        self.features = [list(set(index)) for index in self.tokens[-10:]]
        self.labels = [0 if index == "negative" else 1 for index in self.sentiments[-10:]]

    def encode(self, text):
        words = self.clean_text(text.lower()).split()
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        return " ".join([self.index2word[index] for index in tokens])

## 验证

### 测试

In [5]:
dataset = IMDBDataset('tinyimdb.csv')
print('Vocabulary count:', len(dataset.vocabulary))
print('Review: ', dataset.reviews[0])
print('Tokens: ', dataset.tokens[0])

message = 'i recommend this film'
print('Message: ', message)

tokens = dataset.encode(message)
print('Encode: ', tokens)
print('Decode: ', dataset.decode(tokens))

Vocabulary count: 86
Review:  this movie was excellent. i enjoyed the plot and acting. the character was wonderful. recommend. screenplay actor actress by is a
Tokens:  [61, 69, 20, 42, 56, 47, 11, 54, 73, 65, 11, 9, 20, 44, 66, 38, 52, 74, 32, 82, 79]
Message:  i recommend this film
Encode:  [56, 66, 61, 58]
Decode:  i recommend this film
